This code was originally written by Ryan Serpico. Many thanks to his dedication to documentation. 

Additionally, if you are having issues with the notebook picking up on the `dataretrieval` module, try the solution outlined in [this stackoverflow issue](https://stackoverflow.com/questions/54979928/module-not-found-in-jupyter-lab-but-works-fine-in-jupyter-notebook).

# Collect & clean data

This notebook is dedicated to collecting, cleaning and analyzing all the data that we need for our story. This includes:
- Historical water level data for the Edwards Aquifer.
- Historical fullness data for several nearby lakes.
- Historical discharge data for several nearby rivers.

I will list out the sources of the data as we go along.

## Imports

Let's start our project by importing all the libraries we need.

In [1]:
# To automate the collection of certain datasets, we'll use beautifulsoup4 and requests
import requests
from bs4 import BeautifulSoup

# For data manipulation
import pandas as pd

# We use this library to streamline the process of downloading data from the USGS
import dataretrieval.nwis as nwis

/Users/alexandra.kanik/CHRON/projects/_2023/sa-water-levels/sa-water-levels/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Edwards Aquifer water levels

The first dataset we'll collect and clean is the Edwards Aquifer Authority's J17 aquifer data [from their historical data page](https://www.edwardsaquifer.org/science-maps/aquifer-data/historical-data/).

In [2]:
edwards_aquifer_authority_url = 'https://www.edwardsaquifer.org/science-maps/aquifer-data/historical-data/'

def getSoup(url):
    page = requests.get(url) 
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

soup = getSoup(edwards_aquifer_authority_url)

# Find the href that contains the word 'csv'
csv_links = soup.find_all('a', href=lambda href: href and 'csv' in href)
j17_historical_data = csv_links[0]['href']

# Import j17_historical_data into a pandas dataframe
j17_historical_data_df = pd.read_csv(j17_historical_data)

j17_historical_data_df.head()

,Site,DailyHighDate,WaterLevelElevation
0,J17WL,2024-01-07,640.12
1,J17WL,2024-01-06,640.28
2,J17WL,2024-01-05,640.08
3,J17WL,2024-01-04,639.85
4,J17WL,2024-01-03,639.82


Alright, we successfully used beautiful soup to grab the J17 data off of the historical data page and imported it into a pandas dataframe. Now we'll clean it up a bit.

In [3]:
# Let's convert the "DailyHighDate" column to a datetime object
j17_historical_data_df['DailyHighDate'] = pd.to_datetime(j17_historical_data_df['DailyHighDate'])

# Let's create a new column titled "Year" that contains the year of the "DailyHighDate" column
j17_historical_data_df['Year'] = j17_historical_data_df['DailyHighDate'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "DailyHighDate" column, 
#but replaces the year with 2050. We're doing all of this to make it easier for us to visualize it in a 
#Datawrapper line chart in the future. The 2050 year will not appear in the final product and has no bearing on our analysis.
j17_historical_data_df['dw_date'] = j17_historical_data_df['DailyHighDate'].dt.strftime('%m/%d') + '/2050'

# Let's filter the dataframe to only include years from 1980 to 2024.
j17_historical_data_df = j17_historical_data_df[(j17_historical_data_df['Year'] >= 2000) & (j17_historical_data_df['Year'] <= 2024)].reset_index()

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = j17_historical_data_df[j17_historical_data_df['DailyHighDate'] == j17_historical_data_df['DailyHighDate'].max()]['dw_date'].values[0]

# Let's filter the dataframe to only include rows where the dw_date is between 2023-01-01 and our latest date
#j17_historical_data_df = j17_historical_data_df[(j17_historical_data_df['dw_date'] >= '01/01/2023') & (j17_historical_data_df['dw_date'] <= latest_date)].reset_index()

# Pivot the j17_historical_data_df so that each year is a column and dw_date is the index with the values being the "DailyHigh" column. This is so that we can easily visualize it in a Datawrapper line chart.
j17_historical_data_df = j17_historical_data_df.pivot(index='dw_date', columns='Year', values='WaterLevelElevation').reset_index()

# Export j17_historical_data_df to a csv file
j17_historical_data_df.to_csv('../output/aquifers/j17_historical_data.csv', index=False)

# Let's take a look at the first 5 rows of the dataframe
display(j17_historical_data_df.head())

# Let's take a look at the last 5 rows of the dataframe
display(j17_historical_data_df.tail())

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,01/01/2050,663.46,676.72,684.80,694.75,679.44,697.33,NaN,666.10,689.19,...,633.79,666.70,685.35,666.10,686.42,671.94,663.78,663.84,637.61,639.30
1,01/02/2050,663.90,676.59,684.68,694.64,679.44,697.29,NaN,666.00,689.10,...,634.04,666.37,685.41,665.94,686.20,672.13,663.88,663.63,637.40,639.16
2,01/03/2050,663.92,676.38,684.40,694.41,679.42,697.02,677.49,665.84,688.98,...,634.41,666.69,685.51,665.62,686.32,671.81,664.03,663.25,637.16,639.82
3,01/04/2050,663.56,676.40,684.22,694.35,679.27,697.03,677.13,666.96,688.88,...,634.73,667.70,685.12,665.09,686.50,671.69,663.86,663.02,636.72,639.85
4,01/05/2050,663.44,676.45,684.51,694.43,678.96,697.04,677.09,667.88,688.99,...,634.93,667.55,684.81,664.44,686.72,671.66,663.82,662.89,636.41,640.08


Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
361,12/27/2050,676.46,685.22,694.89,679.86,697.9,679.04,665.44,690.27,670.43,...,666.73,685.84,666.78,685.71,672.39,663.30,664.70,636.41,639.44,NaN
362,12/28/2050,676.54,685.04,694.79,679.86,697.8,678.80,665.41,690.17,670.21,...,666.76,685.63,666.65,685.71,672.21,663.00,664.44,637.01,639.01,NaN
363,12/29/2050,676.51,684.96,694.80,679.61,697.6,678.60,665.39,689.94,669.89,...,666.71,685.61,666.30,685.77,672.55,662.93,663.99,636.98,639.12,NaN
364,12/30/2050,676.60,684.92,694.86,679.24,697.4,678.30,665.63,689.75,669.62,...,666.68,685.48,666.24,685.93,672.07,662.85,663.83,636.93,639.24,NaN
365,12/31/2050,676.72,684.85,694.60,679.16,697.3,678.13,666.04,689.60,669.51,...,666.91,685.27,666.46,686.11,671.81,663.32,663.59,637.29,639.40,NaN


We are also going ot do a 20-year daily average between 2000 and 2020. This is going to help us show that the last few years are been absurdly low... I think.

In [4]:
j17_historical_data_df['20-yr avg.'] = j17_historical_data_df.iloc[:,1:21].mean(axis=1)
j17_avg = j17_historical_data_df.drop(labels=list(j17_historical_data_df.iloc[:,1:21].columns),axis=1)

j17_avg.to_csv('../output/aquifers/avg/j17_historical_data_avg.csv', index=False)
display(j17_avg.head())
display(j17_avg.tail())

Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,671.94,663.78,663.84,637.61,639.30,670.884737
1,01/02/2050,672.13,663.88,663.63,637.40,639.16,670.872105
2,01/03/2050,671.81,664.03,663.25,637.16,639.82,671.138000
3,01/04/2050,671.69,663.86,663.02,636.72,639.85,671.157500
4,01/05/2050,671.66,663.82,662.89,636.41,640.08,671.216500


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
361,12/27/2050,663.30,664.70,636.41,639.44,NaN,671.8890
362,12/28/2050,663.00,664.44,637.01,639.01,NaN,671.7575
363,12/29/2050,662.93,663.99,636.98,639.12,NaN,671.7145
364,12/30/2050,662.85,663.83,636.93,639.24,NaN,671.6945
365,12/31/2050,663.32,663.59,637.29,639.40,NaN,671.6590


## The lakes

Now that we have the Edwards Aquifer data, we'll grab the water level data for several nearby lakes. The lakes we'll be using are:
- Medina Lake
- Canyon Lake
- Lake Travis

We're sourcing this data from [waterdatafortexas.org](https://waterdatafortexas.org/reservoirs/statewide), a product from [the Texas Water Development Board](https://www.twdb.texas.gov/). Their mission "is to lead the state's efforts in ensuring a secure water future for Texas and its citizens."

In [5]:
# Let's import the lake data from the waterdatafortexas.org website. We don't need to use BeautifulSoup for this because the data appears to be located at a static URL. We skip rows because there's a bunch of metadata at the top of the CSV file that we don't need.
medina_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/medina.csv", skiprows=55)
canyon_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/canyon.csv", skiprows=54)
travis_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/travis.csv", skiprows=53)

# I'll print out the head of medina_lake_full_data_df so that we can see what it looks like ahead of time.
travis_lake_full_data_df.head()

,date,water_level,surface_area,reservoir_storage,conservation_storage,percent_full,conservation_capacity,dead_pool_capacity
0,1940-09-30,550.7,1864.0,44232,22807,2.0,1113531,21425
1,1940-10-31,548.2,1750.0,39722,18297,1.6,1113531,21425
2,1940-11-30,595.5,5209.0,194535,173110,15.5,1113531,21425
3,1940-12-31,615.0,7320.0,315950,294525,26.4,1113531,21425
4,1941-01-31,614.0,7205.0,308688,287263,25.8,1113531,21425


In [6]:
# I want to have all the lakes in a single dataframe so that we can easily clean all of the data at once. I'll create a new dataframe called "lake_data_df" that contains all of the data from the other dataframes. Before exporting the data to a CSV file, I'll break it up into individual dataframes again.

# Before we do that, let's create a new column called "Lake" that contains the name of the lake. We'll use this column to filter the data later on.
medina_lake_full_data_df['Lake'] = 'Medina'
canyon_lake_full_data_df['Lake'] = 'Canyon'
travis_lake_full_data_df['Lake'] = 'Travis'

# Let's create a new dataframe called "lake_data_df" that contains all of the data from the other dataframes.
lake_data_df = pd.concat([medina_lake_full_data_df, canyon_lake_full_data_df, travis_lake_full_data_df])

# Let's take a look at the first 5 rows of the dataframe
lake_data_df.head()

,date,water_level,surface_area,reservoir_storage,conservation_storage,percent_full,conservation_capacity,dead_pool_capacity,Lake
0,1997-08-09,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
1,1997-08-10,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
2,1997-08-11,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
3,1997-08-12,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
4,1997-08-13,1071.9,6653.70,303783,254823,100.0,254823,0,Medina


In [7]:
# Alright, let's do some cleaning.

# Let's convert the date column of each dataframe to a datetime object
lake_data_df['date'] = pd.to_datetime(lake_data_df['date'])

# Let's create a new column titled "Year" that contains the year of the "date" column
# medina_lake_full_data_df['Year'] = medina_lake_full_data_df['date'].dt.year
lake_data_df['Year'] = lake_data_df['date'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "date" column, 
#but replaces the year with 2050. We're doing all of this to make it easier for us to visualize it 
#in a Datawrapper line chart in the future. The 2050 year will not appear in the final product and 
#has no bearing on our analysis.
lake_data_df['dw_date'] = lake_data_df['date'].dt.strftime('%m/%d') + '/2050'

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = lake_data_df[lake_data_df['date'] == lake_data_df['date'].max()]['dw_date'].iloc[0]

# Let's only keep records where the "Year" column in the lake_data_df is greater than or equal to 2000
lake_data_df = lake_data_df[lake_data_df['Year'] >= 2000]

# Let's filter the dataframe to only include rows where the dw_date is between 2050-01-01 and the latest date in the dataset
#lake_data_df = lake_data_df[(lake_data_df['dw_date'] >= '01/01/2050') & (lake_data_df['dw_date'] <= latest_date)].reset_index()

# Let's break up the lake_data_df dataframe into individual dataframes again.
medina_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Medina']
canyon_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Canyon']
travis_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Travis']

# Pivot the medina_lake_full_data_df so that each year is a column and dw_date is the index with the values being the "percent_full" column. This is so that we can easily visualize it in a Datawrapper line chart.
medina_lake_full_data_df = medina_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()
canyon_lake_full_data_df = canyon_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()
travis_lake_full_data_df = travis_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()

# Export each of the dataframes to a csv file
medina_lake_full_data_df.to_csv('../output/lakes/medina_lake_full_data.csv', index=False)
canyon_lake_full_data_df.to_csv('../output/lakes/canyon_lake_full_data.csv', index=False)
travis_lake_full_data_df.to_csv('../output/lakes/travis_lake_full_data.csv', index=False)

display(canyon_lake_full_data_df.head())
display(canyon_lake_full_data_df.tail())

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,01/01/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,75.5,100.0,100.0,92.7,100.0,93.4,89.0,99.4,79.6,60.3
1,01/02/2050,93.3,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,75.5,100.0,100.0,92.7,100.0,93.4,88.9,99.4,79.6,60.3
2,01/03/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,75.6,100.0,100.0,92.7,100.0,93.4,88.9,99.3,79.6,60.3
3,01/04/2050,93.4,100.0,100.0,100.0,99.8,100.0,95.2,85.1,100.0,...,75.6,100.0,100.0,92.6,100.0,93.4,88.9,99.2,79.5,60.3
4,01/05/2050,93.3,100.0,100.0,99.7,99.8,100.0,95.2,85.1,100.0,...,75.5,100.0,100.0,92.6,100.0,93.3,88.9,99.2,79.4,60.3


Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
361,12/27/2050,100.0,100.0,100.0,99.7,100.0,95.3,84.9,100.0,78.0,...,100.0,100.0,92.9,100.0,93.5,88.9,99.5,79.8,60.6,NaN
362,12/28/2050,100.0,100.0,100.0,99.7,100.0,95.2,84.8,100.0,78.0,...,100.0,100.0,92.9,100.0,93.5,88.9,99.5,79.8,60.5,NaN
363,12/29/2050,100.0,100.0,100.0,99.7,100.0,95.2,84.9,100.0,77.9,...,100.0,100.0,92.8,100.0,93.5,88.8,99.5,79.7,60.5,NaN
364,12/30/2050,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,77.9,...,100.0,100.0,92.8,100.0,93.4,88.9,99.5,79.7,60.4,NaN
365,12/31/2050,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,77.9,...,100.0,100.0,92.8,100.0,93.4,89.0,99.5,79.7,60.4,NaN


In [8]:
medina_lake_full_data_df['20-yr avg.'] = medina_lake_full_data_df.iloc[:,1:21].mean(axis=1)
medina_lake_avg = medina_lake_full_data_df.drop(labels=list(medina_lake_full_data_df.iloc[:,1:21].columns),axis=1)

medina_lake_avg.to_csv('../output/aquifers/avg/jmedina_lake_data_avg.csv', index=False)
display(medina_lake_avg.head())
display(medina_lake_avg.tail())

Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,78.0,41.7,25.9,6.3,3.3,62.820
1,01/02/2050,77.9,41.6,25.9,6.3,3.3,62.825
2,01/03/2050,77.8,41.6,25.8,6.3,3.3,62.795
3,01/04/2050,77.7,41.5,25.8,6.3,3.3,62.775
4,01/05/2050,77.6,41.4,25.7,6.3,3.3,62.750


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
361,12/27/2050,41.8,26.0,6.4,3.4,NaN,62.970
362,12/28/2050,41.8,26.0,6.3,3.4,NaN,62.950
363,12/29/2050,41.8,25.9,6.3,3.4,NaN,62.925
364,12/30/2050,41.7,25.9,6.3,3.4,NaN,62.890
365,12/31/2050,41.7,25.9,6.3,3.4,NaN,62.860


In [9]:
canyon_lake_full_data_df['20-yr avg.'] = canyon_lake_full_data_df.iloc[:,1:21].mean(axis=1)
canyon_lake_avg = canyon_lake_full_data_df.drop(labels=list(canyon_lake_full_data_df.iloc[:,1:21].columns),axis=1)

canyon_lake_avg.to_csv('../output/aquifers/avg/canyon_lake_data_avg.csv', index=False)
display(canyon_lake_full_data_df.head())
display(canyon_lake_avg.head())
display(canyon_lake_avg.tail())

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,100.0,100.0,92.7,100.0,93.4,89.0,99.4,79.6,60.3,92.145
1,01/02/2050,93.3,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,100.0,100.0,92.7,100.0,93.4,88.9,99.4,79.6,60.3,92.130
2,01/03/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,100.0,100.0,92.7,100.0,93.4,88.9,99.3,79.6,60.3,92.135
3,01/04/2050,93.4,100.0,100.0,100.0,99.8,100.0,95.2,85.1,100.0,...,100.0,100.0,92.6,100.0,93.4,88.9,99.2,79.5,60.3,92.140
4,01/05/2050,93.3,100.0,100.0,99.7,99.8,100.0,95.2,85.1,100.0,...,100.0,100.0,92.6,100.0,93.3,88.9,99.2,79.4,60.3,92.115


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,93.4,89.0,99.4,79.6,60.3,92.145
1,01/02/2050,93.4,88.9,99.4,79.6,60.3,92.130
2,01/03/2050,93.4,88.9,99.3,79.6,60.3,92.135
3,01/04/2050,93.4,88.9,99.2,79.5,60.3,92.140
4,01/05/2050,93.3,88.9,99.2,79.4,60.3,92.115


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
361,12/27/2050,88.9,99.5,79.8,60.6,NaN,92.180
362,12/28/2050,88.9,99.5,79.8,60.5,NaN,92.165
363,12/29/2050,88.8,99.5,79.7,60.5,NaN,92.165
364,12/30/2050,88.9,99.5,79.7,60.4,NaN,92.155
365,12/31/2050,89.0,99.5,79.7,60.4,NaN,92.150


In [10]:
travis_lake_full_data_df['20-yr avg.'] = travis_lake_full_data_df.iloc[:,1:21].mean(axis=1)
travis_lake_avg = travis_lake_full_data_df.drop(labels=list(travis_lake_full_data_df.iloc[:,1:21].columns),axis=1)

travis_lake_avg.to_csv('../output/aquifers/avg/travis_lake_data_avg.csv', index=False)
display(travis_lake_full_data_df.head())
display(travis_lake_avg.head())
display(travis_lake_avg.tail())

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,73.8,100.0,99.6,100.0,79.2,100.0,73.5,50.4,99.3,...,92.8,100.0,81.2,100.0,81.5,67.3,70.9,45.8,37.6,74.785
1,01/02/2050,73.8,100.0,100.0,99.9,79.1,100.0,73.5,50.5,99.5,...,92.9,100.0,81.2,100.0,81.7,67.3,70.9,45.7,37.6,74.810
2,01/03/2050,73.8,100.0,100.0,100.0,79.1,100.0,73.5,50.5,99.9,...,93.0,100.0,81.3,100.0,82.3,67.3,70.8,45.7,37.8,74.845
3,01/04/2050,73.7,100.0,100.0,100.0,79.1,100.0,73.5,50.7,100.0,...,93.1,100.0,81.3,100.0,83.0,67.3,70.7,45.7,37.8,74.865
4,01/05/2050,73.7,100.0,100.0,100.0,79.2,100.0,73.6,50.7,100.0,...,93.2,100.0,81.4,100.0,83.5,67.2,70.7,45.7,37.8,74.900


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,81.5,67.3,70.9,45.8,37.6,74.785
1,01/02/2050,81.7,67.3,70.9,45.7,37.6,74.810
2,01/03/2050,82.3,67.3,70.8,45.7,37.8,74.845
3,01/04/2050,83.0,67.3,70.7,45.7,37.8,74.865
4,01/05/2050,83.5,67.2,70.7,45.7,37.8,74.900


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
361,12/27/2050,66.9,71.0,45.9,37.7,NaN,75.200
362,12/28/2050,66.9,70.9,45.8,37.7,NaN,75.195
363,12/29/2050,66.8,70.9,45.8,37.7,NaN,75.190
364,12/30/2050,66.9,70.9,45.8,37.6,NaN,75.190
365,12/31/2050,67.2,70.9,45.8,37.6,NaN,75.170


## The rivers

Now that we have the Edwards Aquifer data and the lake data, we'll grab the discharge data for several nearby rivers.

The data are coming from the USGS [National Water Information System](https://waterdata.usgs.gov/nwis/rt) (NWIS) through their [dataretrieval-python library](https://github.com/DOI-USGS/dataretrieval-python).

 The rivers we'll be using are:
- Guadalupe River at Spring Branch, Texas
- Guadalupe River at Sattler, Texas
- Guadalupe River at New Braunfels, Texas
- San Marcos River in San Marcos, Texas
- Blanco River at Wimberley, Texas
- Medina River at Patterson Road in Medina, Texas
- Medina River at La Coste, Texas
- San Antonio River near Floresvilles, Texas

In [11]:
# Let's start by creating a dictionary that contains the name of the rivers we're interested in and its site number. We'll use this dictionary to loop through the data and create a dataframe for each river.
rivers = {
    'Guadalupe Rv nr Spring Branch': '08167500',
    'Guadalupe Rv at Sattler': '08167800',
    'Guadalupe Rv Abv Comal Rv at New Braunfels': '08168500',
    'San Marcos Rv at San Marcos': '08170500',
    'Blanco Rv at Wimberley': '08171000',
    'Medina Rv at Patterson Rd at Medina': '0817887350',
    'Medina Rv at La Coste': '08180640',
    'San Antonio Rv nr Floresville': '08183200',
}

# Convert the values in the dictionary to a list
rivers = list(rivers.values())

unified_rivers_df = nwis.get_record(sites=rivers, service='dv', start='2000-01-01', parameterCd='00060')

# Reset the index so that site_no is a column
unified_rivers_df = unified_rivers_df.reset_index()

# Create a new column called "River" that contains the name of the river based on the site number. Refer to the dictionary above to see which site number corresponds to which river.
unified_rivers_df['River'] = unified_rivers_df['site_no'].map({
    '08167500': 'Guadalupe Rv nr Spring Branch',
    '08167800': 'Guadalupe Rv at Sattler',
    '08168500': 'Guadalupe Rv Abv Comal Rv at New Braunfels',
    '08170500': 'San Marcos Rv at San Marcos',
    '08171000': 'Blanco Rv at Wimberley',
    '0817887350': 'Medina Rv at Patterson Rd at Medina',
    '08180640': 'Medina Rv at La Coste',
    '08183200': 'San Antonio Rv nr Floresville',
})

unified_rivers_df.to_csv('../output/rivers/unified_rivers.csv', index=False)

# Let's reorganize the columns so that the "River" column is the first column
unified_rivers_df = unified_rivers_df[['River', 'site_no', 'datetime', '00060_Mean']]

# Rename 00060_Maximum to "streamflow"
unified_rivers_df = unified_rivers_df.rename(columns={'00060_Mean': 'streamflow'})

display(unified_rivers_df.head())
display(unified_rivers_df.tail())

/Users/alexandra.kanik/CHRON/projects/_2023/sa-water-levels/sa-water-levels/env/lib/python3.9/site-packages/dataretrieval/nwis.py:1174: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  record_df = pd.read_json(record_json,
/Users/alexandra.kanik/CHRON/projects/_2023/sa-water-levels/sa-water-levels/env/lib/python3.9/site-packages/dataretrieval/nwis.py:1174: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  record_df = pd.read_json(record_json,
/Users/alexandra.kanik/CHRON/projects/_2023/sa-water-levels/sa-water-levels/env/lib/python3.9/site-packages/dataretrieval/nwis.py:1174: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  rec

,River,site_no,datetime,streamflow
0,Guadalupe Rv nr Spring Branch,08167500,2000-01-01 00:00:00+00:00,86.0
1,Guadalupe Rv nr Spring Branch,08167500,2000-01-02 00:00:00+00:00,89.0
2,Guadalupe Rv nr Spring Branch,08167500,2000-01-03 00:00:00+00:00,89.0
3,Guadalupe Rv nr Spring Branch,08167500,2000-01-04 00:00:00+00:00,86.0
4,Guadalupe Rv nr Spring Branch,08167500,2000-01-05 00:00:00+00:00,88.0


,River,site_no,datetime,streamflow
57674,San Antonio Rv nr Floresville,08183200,2024-01-23 00:00:00+00:00,3200.0
57675,San Antonio Rv nr Floresville,08183200,2024-01-24 00:00:00+00:00,2970.0
57676,San Antonio Rv nr Floresville,08183200,2024-01-25 00:00:00+00:00,4220.0
57677,San Antonio Rv nr Floresville,08183200,2024-01-26 00:00:00+00:00,4600.0
57678,San Antonio Rv nr Floresville,08183200,2024-01-27 00:00:00+00:00,NaN


In [12]:
# Convert the datetime column to a datetime object
unified_rivers_df['datetime'] = pd.to_datetime(unified_rivers_df['datetime'])

# Create a new column called "Year" that contains the year of the datetime column
unified_rivers_df['Year'] = unified_rivers_df['datetime'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "date" column, 
#but replaces the year with 2050. We're doing all of this to make it easier for us to visualize 
#it in a Datawrapper line chart in the future. The 2050 year will not appear in the final product 
#and has no bearing on our analysis.
unified_rivers_df['dw_date'] = unified_rivers_df['datetime'].dt.strftime('%m/%d') + '/2050'

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = unified_rivers_df[unified_rivers_df['datetime'] == unified_rivers_df['datetime'].max()]['dw_date'].iloc[0]

# Let's filter the dataframe to only contain rows where the Year is greater than or equal to 2000
unified_rivers_df = unified_rivers_df[unified_rivers_df['Year'] >= 2000]

# Let's filter the dataframe to only include rows where the dw_date is between 2050-01-01 and the latest date in the dataset
#unified_rivers_df = unified_rivers_df[(unified_rivers_df['dw_date'] >= '01/01/2050') & (unified_rivers_df['dw_date'] <= latest_date)].reset_index(drop=True)

# Break up the dataframe into separate csv files for each River
for river in unified_rivers_df['River'].unique():
    river_df = unified_rivers_df[unified_rivers_df['River'] == river]

    # Pivot the dataframe so that each year is a column and dw_date is the index with the values 
    # being the "streamflow" column. This is so that we can easily visualize it in a Datawrapper line chart.
    river_df = river_df.pivot(index='dw_date', columns='Year', values='streamflow').reset_index()
    # If a row has any missing values, drop it
    # river_df = river_df.dropna()

    # Replace spaces with underscores in the file name
    river = river.replace(' ', '_')
    river_df.to_csv(f'../output/rivers/{river}.csv', index=False)
    display(river)
    display(river_df.head())
    display(river_df.tail())

    # And we do the averages
    loc_2019 = river_df.columns.get_loc(int('2019'))+1
    river_df['20-yr avg.'] = river_df.iloc[:,1:loc_2019].mean(axis=1)
    river_df_avg = river_df.drop(labels=list(river_df.iloc[:,1:loc_2019].columns),axis=1)

    river_df_avg.to_csv(f'../output/aquifers/avg/{river}_data_avg.csv', index=False)
    display(river_df_avg.head())

# unified_rivers_df.to_csv('../output/rivers/river_data.csv')
# unified_rivers_df.head()

'Guadalupe_Rv_nr_Spring_Branch'

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,01/01/2050,86.0,463.0,517.0,615.0,139.0,693.0,109.0,75.0,224.0,...,38.4,312.0,231.0,82.3,734.0,81.2,52.1,89.3,22.2,20.9
1,01/02/2050,89.0,449.0,504.0,572.0,140.0,680.0,110.0,72.8,220.0,...,43.6,307.0,235.0,81.8,737.0,79.1,62.7,84.9,22.9,22.0
2,01/03/2050,89.0,438.0,493.0,558.0,145.0,718.0,113.0,73.8,217.0,...,51.7,322.0,233.0,81.8,939.0,79.2,81.6,80.3,22.8,25.5
3,01/04/2050,86.0,435.0,487.0,545.0,146.0,770.0,112.0,116.0,219.0,...,49.4,355.0,228.0,80.5,903.0,79.3,83.9,81.6,22.0,23.0
4,01/05/2050,88.0,435.0,561.0,543.0,144.0,703.0,112.0,112.0,223.0,...,46.5,363.0,224.0,82.6,839.0,80.2,77.1,79.2,23.7,22.4


Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
361,12/27/2050,625.0,579.0,636.0,142.0,750.0,119.0,93.5,226.0,62.6,...,322.0,258.0,88.4,1400.0,81.8,50.2,86.9,21.1,26.6,NaN
362,12/28/2050,595.0,573.0,621.0,144.0,743.0,119.0,86.4,229.0,62.7,...,335.0,255.0,88.2,1200.0,82.9,50.9,86.1,19.9,32.9,NaN
363,12/29/2050,528.0,562.0,624.0,143.0,734.0,120.0,81.5,225.0,61.7,...,376.0,248.0,89.3,920.0,84.7,49.1,85.7,21.1,25.7,NaN
364,12/30/2050,490.0,545.0,626.0,141.0,723.0,115.0,83.9,226.0,61.0,...,353.0,239.0,90.2,816.0,82.9,48.3,84.8,21.9,22.9,NaN
365,12/31/2050,472.0,530.0,620.0,139.0,710.0,120.0,74.7,229.0,58.9,...,325.0,237.0,86.4,775.0,84.3,50.7,84.5,22.9,22.5,NaN


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,81.2,52.1,89.3,22.2,20.9,238.495
1,01/02/2050,79.1,62.7,84.9,22.9,22.0,233.935
2,01/03/2050,79.2,81.6,80.3,22.8,25.5,245.325
3,01/04/2050,79.3,83.9,81.6,22.0,23.0,248.270
4,01/05/2050,80.2,77.1,79.2,23.7,22.4,245.520


'Guadalupe_Rv_at_Sattler'

...

Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,74.5,35.9,17.5,2.74,0.0,81.866667
1,01/02/2050,75.5,33.5,15.7,2.59,0.0,83.533333
2,01/03/2050,76.1,32.1,15.4,2.78,0.0,85.000000
3,01/04/2050,75.9,31.7,15.1,2.38,0.0,83.266667
4,01/05/2050,72.7,31.2,16.0,2.29,0.0,82.600000


'San_Antonio_Rv_nr_Floresville'

Year,dw_date,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,01/01/2050,NaN,423.0,542.0,179.0,328.0,330.0,208.0,171.0,225.0,199.0,277.0,680.0,294.0,667.0,306.0,448.0,225.0,179.0,174.0
1,01/02/2050,NaN,293.0,615.0,175.0,292.0,296.0,198.0,209.0,208.0,191.0,268.0,670.0,288.0,624.0,297.0,329.0,183.0,178.0,169.0
2,01/03/2050,NaN,298.0,580.0,175.0,283.0,282.0,198.0,236.0,213.0,207.0,925.0,674.0,297.0,869.0,305.0,257.0,177.0,182.0,197.0
3,01/04/2050,393.0,589.0,538.0,187.0,283.0,324.0,211.0,215.0,220.0,568.0,1520.0,664.0,289.0,1040.0,324.0,247.0,167.0,210.0,396.0
4,01/05/2050,353.0,1290.0,527.0,212.0,311.0,304.0,171.0,232.0,219.0,341.0,858.0,656.0,250.0,791.0,300.0,245.0,173.0,212.0,252.0


Year,dw_date,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
361,12/27/2050,360.0,502.0,174.0,289.0,317.0,223.0,154.0,148.0,181.0,230.0,716.0,286.0,719.0,292.0,212.0,277.0,161.0,200.0,NaN
362,12/28/2050,276.0,509.0,184.0,281.0,306.0,228.0,156.0,186.0,188.0,463.0,820.0,298.0,947.0,302.0,220.0,284.0,156.0,181.0,NaN
363,12/29/2050,224.0,511.0,182.0,277.0,319.0,222.0,162.0,200.0,211.0,579.0,766.0,297.0,886.0,300.0,225.0,287.0,161.0,177.0,NaN
364,12/30/2050,264.0,521.0,190.0,356.0,363.0,215.0,162.0,172.0,247.0,347.0,701.0,297.0,743.0,303.0,224.0,278.0,177.0,174.0,NaN
365,12/31/2050,847.0,519.0,182.0,394.0,343.0,211.0,161.0,203.0,207.0,290.0,683.0,298.0,654.0,304.0,235.0,223.0,184.0,171.0,NaN


Year,dw_date,2020,2021,2022,2023,2024,20-yr avg.
0,01/01/2050,306.0,448.0,225.0,179.0,174.0,347.923077
1,01/02/2050,297.0,329.0,183.0,178.0,169.0,332.846154
2,01/03/2050,305.0,257.0,177.0,182.0,197.0,402.846154
3,01/04/2050,324.0,247.0,167.0,210.0,396.0,502.928571
4,01/05/2050,300.0,245.0,173.0,212.0,252.0,465.357143


## Playground

This section is for playing around with the data. It's not part of the collection of data, but it's useful for exploring.

In [ ]:
# Find the median of each column. Put the results in a new dataframe
j17_historical_data_median_df = j17_historical_data_df.median().to_frame().reset_index()

# Sort the dataframe by the median values
j17_historical_data_median_df = j17_historical_data_median_df.sort_values(by=0, ascending=False).reset_index()

j17_historical_data_median_df

In [ ]:
unified_rivers_df.groupby('River')['datetime'].min()

In [ ]:
med_coste = unified_rivers_df.loc[unified_rivers_df['River'] == 'Medina Rv at La Coste']
med_coste_pivot = med_coste.pivot(index='dw_date', columns='Year', values='streamflow').reset_index()

In [ ]:
med_coste_pivot